In [1]:
def quicksort(a, p, r):
    if p < r:
        q = partition(a, p, r)
        quicksort(a, p, q - 1)
        quicksort(a, q + 1, r)

def partition(a, p, r):
    x = a[r]
    i = p - 1
    for j in range(p, r):
        if a[j] <= x:
            i += 1
            a[i], a[j] = a[j], a[i]
    a[i + 1], a[r] = a[r], a[i + 1]
    return i + 1

In [6]:
a = [13,19,9,5,12,8,7,4,21,2,6,11]
quicksort(a, 0, len(a)-1)
print(a)

[2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19, 21]


In [8]:
"""
partition descreasing order
exact same except >= compare on pivot
"""
def partition_desc(a, p, r):
    pivot = a[r]
    i = p - 1

    for j in range(p, r):
        if a[j] >= pivot:
            i += 1
            a[i], a[j] = a[j], a[i]

    a[i + 1], a[r] = a[r], a[i + 1]
    return i + 1

In [9]:
"""
We analyze the expected running time of randomized algorithms because:

Worst-case behavior is usually rare
Expected time reflects typical performance
Randomization protects against bad inputs
It gives strong, input-independent guarantees
"""
import random

def randomized_partition(a, p, r):
    i = random.randint(p, r)
    a[r], a[i] = a[i], a[r]
    return partition(a, p, r)

def randomized_quicksort(a, p, r):
    if p < r:
        q = randomized_partition(a, p, r)
        randomized_quicksort(a, p, q - 1)
        randomized_quicksort(a, q + 1, r)

In [13]:
"""
This performs better than the partition above!

Although using 3-way partitioning (Dutch National Flag) we can always improve
"""
def hoare_partition(a, left, right):
    pivot = a[left]
    i, j = left - 1, right + 1

    while True:
        i += 1
        while a[i] < pivot:
            i += 1

        j -= 1
        while a[j] > pivot:
            j -= 1

        if i >= j:
            return j

        a[i], a[j] = a[j], a[i]

In [12]:
a = [13, 19, 9, 5, 12, 8, 7, 4, 11, 2, 6, 21]
hoare_partition(a, 0, len(a)-1)
a

[6, 2, 9, 5, 12, 8, 7, 4, 11, 19, 13, 21]

In [14]:
def quicksort(a, p, r):
    if p < r:
        q = hoare_partition(a, p, r)
        quicksort(a, p, q)
        quicksort(a, q + 1, r)

In [17]:
"""
Dutch National Flag partition
"""
def partition_prime(a, p, r):
    """
    Rearranges a[p:r+1] and returns (q, t) such that:
      a[p:q]   < pivot
      a[q:t+1] == pivot
      a[t+1:r] > pivot
    """
    pivot = a[r]
    i = p # < pviot
    k = p # current index
    j = r # > pivot

    while k <= j:
        if a[k] < pivot:
            a[i], a[k] = a[k], a[i]
            i += 1
            k += 1
        elif a[k] > pivot:
            a[k], a[j] = a[j], a[k]
            j -= 1
        else:
            k += 1
    return i, j

In [18]:
a = [3, 5, 2, 3, 3, 1, 5, 3]
q, t = partition_prime(a, 0, len(a) - 1)

a
print("q =", q, "t =", t)
print("less than:", a[:q])
print("equal to:", a[q:t+1])
print("greater than:", a[t+1:])

q = 2 t = 5
less than: [2, 1]
equal to: [3, 3, 3, 3]
greater than: [5, 5]


In [19]:
import random

def randomized_partition_prime(a, p, r):
    i = random.randint(p, r)
    a[i], a[r] = a[r], a[i]
    return partition_prime(a, p, r)

In [21]:
"""
skips recursion on equal keys by using a 3-way randomized partition,
eliminating the Θ(n²) behavior caused by duplicates.
"""
def quicksort_prime(a, p=0, r=None):
    if r is None:
        r = len(a) - 1

    if p < r:
        q, t = randomized_partition_prime(a, p, r)
        quicksort_prime(a, p, q - 1)
        quicksort_prime(a, t + 1, r)

In [22]:
def stooge_sort(a, p, r):
    if a[p] > a[r]:
        a[p], a[r] = a[r], a[p]
    if p + 1 < r:
        k = (r - p + 1) // 3
        stooge_sort(a, p, r - k)
        stooge_sort(a, p + k, r)
        stooge_sort(a, p, r - k)

In [23]:
def tre_quicksort(a, p, r):
    while p < r:
        q = partition(a, p, r)
        tre_quicksort(a, p, q - 1)
        p = q + 1

"""
Problem:
If the left partition is always large, you still get deep recursion.

Best practice:
Always recurse on the smaller partition and iterate on the larger one.
This guarantees O(log n) stack depth even in the worst case.
"""
def tre_quicksort(a, p, r):
    while p < r:
        q = partition(a, p, r)

        if q - p < r - q:
            tre_quicksort(a, p, q - 1)
            p = q + 1
        else:
            tre_quicksort(a, q + 1, r)
            r = q - 1

In [24]:
import random

def overlaps(i1, i2):
    return not (i1[1] < i2[0] or i2[1] < i1[0])

def fuzzy_sort(intervals, depth=0):
    indent = "  " * depth

    if len(intervals) <= 1:
        return intervals

    pivot = random.choice(intervals)
    L, R = pivot

    print(f"{indent}Pivot chosen: {pivot}")

    # Compute intersection of all intervals overlapping the pivot
    for a, b in intervals:
        if not (b < pivot[0] or a > pivot[1]):
            L = max(L, a)
            R = min(R, b)

    print(f"{indent}Intersection interval: [{L}, {R}]")

    left, mid, right = [], [], []

    for a, b in intervals:
        if b < L:
            left.append((a, b))
        elif a > R:
            right.append((a, b))
        else:
            mid.append((a, b))

    print(f"{indent}Left:  {left}")
    print(f"{indent}Mid:   {mid}")
    print(f"{indent}Right: {right}\n")

    return (
        fuzzy_sort(left, depth + 1)
        + mid
        + fuzzy_sort(right, depth + 1)
    )

In [25]:
intervals = [
    (6, 10),
    (1, 4),
    (3, 8),
    (2, 6),
    (9, 12)
]